# Join QSO-maker catalogs for Iron
Stephanie Juneau (NOIRLab)


NOTE: Edmond put individual files; need to loop or create a compilation
```
## QSO-maker path 
path_qsom = '/global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/'
```

## Imports

In [ ]:
# General imports
import numpy as np

# Import Astropy libraries - useful for many astronomy related function
from astropy.table import Table, vstack
from astropy.io import fits

# Fast FITS file I/O access
import fitsio


## Data files

In [ ]:
## Define filepaths
path_qsom = '/global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/'
path_gqp = '/global/cfs/cdirs/desi/science/gqp/agncatalog/qsomaker/iron/'

In [ ]:
## SJ: will exclude the targeting cols because we'll add them from the zcat VAC instead 
#qsom_cols=['TARGETID','Z','ZERR','ZWARN','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC',\
#           'MORPHTYPE','EBV','MASKBITS','DESI_TARGET','SCND_TARGET','COADD_NUMEXP','COADD_EXPTIME',\
#           'CMX_TARGET','SV1_DESI_TARGET','SV2_DESI_TARGET','SV3_DESI_TARGET',\
#           'SV1_SCND_TARGET','SV2_SCND_TARGET','SV3_SCND_TARGET','TSNR2_LYA','TSNR2_QSO',\
#           'DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII',\
#           'Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha','QSO_MASKBITS']

# Current choice for Iron
qsom_cols=['TARGETID','Z','ZERR','ZWARN','OBJTYPE','SPECTYPE','COADD_FIBERSTATUS','TARGET_RA','TARGET_DEC',\
           'MORPHTYPE','MASKBITS','COADD_NUMEXP','COADD_EXPTIME','TSNR2_LYA','TSNR2_QSO',\
           'DELTA_CHI2_MGII','A_MGII','SIGMA_MGII','B_MGII','VAR_A_MGII','VAR_SIGMA_MGII','VAR_B_MGII',\
           'Z_RR','Z_QN','C_LYA','C_CIV','C_CIII','C_MgII','C_Hbeta','C_Halpha',\
           'QSO_MASKBITS','SURVEY','PROGRAM']
# Try without these: 'Z_LYA','Z_CIV','Z_CIII','Z_MgII','Z_Hbeta','Z_Halpha'

## NOTE: cut on OBJTYPE then delete the column (wasn't in Fuji/EDR version)

print(qsom_cols)

## Loop over survey-program combos

In [ ]:
%%time
# Short list for testing
#survey_programs = ['cmx_other', \
#                   'special_dark', 'special_other', \
#                   'sv1_backup', 'sv1_bright']

# Complete list for DR1
survey_programs = ['cmx_other', 'main_backup', 'main_bright', 'main_dark',\
                   'special_backup', 'special_bright', 'special_dark', 'special_other', \
                   'sv1_backup', 'sv1_bright', 'sv1_dark', 'sv1_other', \
                   'sv2_backup', 'sv2_bright', 'sv2_dark', \
                   'sv3_backup', 'sv3_bright', 'sv3_dark']

# Initialize table
T_qsom = Table()

for i in range(len(survey_programs)):

    # Read each file
    surv_prog = survey_programs[i]
    file_i = path_qsom+f"QSO_cat_iron_{surv_prog}_healpix_all_targets_v1.fits"
    T_qsom_i = Table(fitsio.read(file_i, columns=qsom_cols, ext=1)) 

    # Print some stats:
    N_init = len(T_qsom_i)
    print(f"N={N_init} in file {file_i}")
    
    ## Keep only OBJTYPE='TGT'
    keep = T_qsom_i['OBJTYPE']=='TGT'
    T_qsom_i = T_qsom_i[keep]
    
    # Print some stats:
    N_keep = len(T_qsom_i)
    print(f"... After cutting on OBTYPE=TGT: N={N_keep}; (fraction: {np.round(N_keep/N_init, 2)})")

    ## Adding two columns we need for the cuts
    a = np.array([T_qsom_i['C_LYA'], T_qsom_i['C_CIV'], T_qsom_i['C_CIII'], \
                  T_qsom_i['C_MgII'], T_qsom_i['C_Hbeta'], T_qsom_i['C_Halpha']])
    T_qsom_i['QN_C_LINE_BEST'] = [max(l) for l in (a.T).tolist()]
    T_qsom_i['QN_C_LINE_SECOND_BEST'] = [sorted(l)[-2] for l in (a.T).tolist()]

    hi_conf_50 = T_qsom_i['QN_C_LINE_BEST']>0.50
    # Remove stars (except wait for possible mid/high-confidence QN cases)
    is_star = (T_qsom_i['SPECTYPE']=='STAR')&(T_qsom_i['Z']<0.001)&(~hi_conf_50)
    T_qsom_i = T_qsom_i[~is_star]
    
    # Print some stats:
    print(f"... After cutting Stars at z<0.001: N={len(T_qsom_i)}; (fraction: {np.round(len(T_qsom_i)/N_init, 2)})")
    
    T_qsom = vstack([T_qsom, T_qsom_i])

In [ ]:
print(len(T_qsom))

T_qsom[:3]

In [ ]:
%%time
#NOTE: this is slow...
#print(np.unique(T_qsom['SURVEY','PROGRAM']))

In [ ]:
print(len(T_qsom))
print(len(T_qsom[T_qsom['COADD_EXPTIME']==0.]))

In [ ]:
T_qsom.remove_column('OBJTYPE')

In [ ]:
%%time
## Save output file
outfile = path_gqp+"QSO_cat_iron_healpix_all_targets_v1.fits"
T_qsom.write(outfile, overwrite=True)

## Record keeping of results from above

Looping over all survey-program combos printed this:
```
N=5000 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_cmx_other_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=4146; (fraction: 0.83)
... After cutting Stars at z<0.001: N=3509; (fraction: 0.7)
N=1632500 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_main_backup_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=1233814; (fraction: 0.76)
... After cutting Stars at z<0.001: N=27723; (fraction: 0.02)
N=11020470 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_main_bright_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=9069114; (fraction: 0.82)
... After cutting Stars at z<0.001: N=6490892; (fraction: 0.59)
N=12778525 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_main_dark_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=10591432; (fraction: 0.83)
... After cutting Stars at z<0.001: N=10155396; (fraction: 0.79)
N=44905 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_special_backup_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=35648; (fraction: 0.79)
... After cutting Stars at z<0.001: N=2894; (fraction: 0.06)
N=74412 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_special_bright_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=57237; (fraction: 0.77)
... After cutting Stars at z<0.001: N=43780; (fraction: 0.59)
N=19500 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_special_dark_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=16076; (fraction: 0.82)
... After cutting Stars at z<0.001: N=15217; (fraction: 0.78)
N=64428 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_special_other_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=53555; (fraction: 0.83)
... After cutting Stars at z<0.001: N=43035; (fraction: 0.67)
N=110599 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv1_backup_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=69665; (fraction: 0.63)
... After cutting Stars at z<0.001: N=10989; (fraction: 0.1)
N=239057 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv1_bright_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=197959; (fraction: 0.83)
... After cutting Stars at z<0.001: N=155744; (fraction: 0.65)
N=371000 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv1_dark_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=311122; (fraction: 0.84)
... After cutting Stars at z<0.001: N=283910; (fraction: 0.77)
N=143679 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv1_other_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=109606; (fraction: 0.76)
... After cutting Stars at z<0.001: N=49329; (fraction: 0.34)
N=4985 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv2_backup_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=3213; (fraction: 0.64)
... After cutting Stars at z<0.001: N=339; (fraction: 0.07)
N=82288 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv2_bright_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=55445; (fraction: 0.67)
... After cutting Stars at z<0.001: N=48384; (fraction: 0.59)
N=85411 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv2_dark_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=59345; (fraction: 0.69)
... After cutting Stars at z<0.001: N=55882; (fraction: 0.65)
N=156359 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv3_backup_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=90685; (fraction: 0.58)
... After cutting Stars at z<0.001: N=6380; (fraction: 0.04)
N=729898 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv3_bright_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=484457; (fraction: 0.66)
... After cutting Stars at z<0.001: N=268049; (fraction: 0.37)
N=862947 in file /global/cfs/cdirs/desi/users/edmondc/QSO_catalog/iron/all/QSO_cat_iron_sv3_dark_healpix_all_targets_v1.fits
... After cutting on OBTYPE=TGT: N=618208; (fraction: 0.72)
... After cutting Stars at z<0.001: N=599194; (fraction: 0.69)
```

## Tests and VI below

In [ ]:
print(170042/18260646)

In [ ]:
%%time
#test
file_i = path_qsom+"QSO_cat_iron_sv1_backup_healpix_all_targets_v1.fits"
#file_i = path_qsom+"QSO_cat_iron_sv3_dark_healpix_all_targets_v1.fits"
#file_i = path_qsom+"QSO_cat_iron_sv2_bright_healpix_all_targets_v1.fits" #N=1
#file_i = path_qsom+"QSO_cat_iron_sv2_dark_healpix_all_targets_v1.fits"

T_qsom_i = Table(fitsio.read(file_i, columns=qsom_cols, ext=1))

## Adding two columns we need for the cuts
a = np.array([T_qsom_i['C_LYA'], T_qsom_i['C_CIV'], T_qsom_i['C_CIII'], \
              T_qsom_i['C_MgII'], T_qsom_i['C_Hbeta'], T_qsom_i['C_Halpha']])
T_qsom_i['QN_C_LINE_BEST'] = [max(l) for l in (a.T).tolist()]
T_qsom_i['QN_C_LINE_SECOND_BEST'] = [sorted(l)[-2] for l in (a.T).tolist()]


In [ ]:
#fits.info(file_i)

In [ ]:
print(np.unique(T_qsom_i['OBJTYPE']))

In [ ]:
for objtyp in np.unique(T_qsom_i['OBJTYPE']):
    print(f"OBJTYPE={objtyp};  N={len(T_qsom_i[T_qsom_i['OBJTYPE']==objtyp])}")

In [ ]:
no_objtype = (T_qsom_i['OBJTYPE']=='')
print(np.max(T_qsom_i['TARGETID'][no_objtype]))
print(np.min(T_qsom_i['ZWARN'][no_objtype]))
print(np.min(T_qsom_i['COADD_FIBERSTATUS'][no_objtype]))

In [ ]:
print(len(T_qsom_i))
print(len(T_qsom_i[T_qsom_i['TARGETID']<0]))
is_star = (T_qsom_i['SPECTYPE']=='STAR')&(T_qsom_i['Z']<0.001)
print(len(T_qsom_i[is_star]))

In [ ]:
hi_conf_95 = T_qsom_i['QN_C_LINE_BEST']>0.5
print(len(T_qsom_i[is_star&hi_conf_95]))

In [ ]:
T_qsom_i[is_star&hi_conf_95]

In [ ]:
prospect_prefix = 'https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid'
#'https://www.legacysurvey.org/viewer-desi/desi-spectrum/daily/targetid'

In [ ]:
for tid in T_qsom_i['TARGETID'][is_star&hi_conf_95][:10]:
    print(prospect_prefix+str(tid))

### SV1 backup (N=4)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39633489959586121 (STAR, RR correct)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39633493742847107 (STAR but artifact/fake break)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39633478597218272 (STAR)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39633554824495772 (STAR but artifact/fake break)

### SV1 bright (N=8)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39627878731613209 (STAR, jump in B/R gap)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39632940052778995 (STAR, jump in B/R gap, fiber off-center?)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39632955525563342 (? looks bad, not sure)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39632995233039010 (QSO at z=2.72 --> LENSED)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39628500637844358 (STAR, artifact in red)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39633134756564663 (STAR, artifact in red)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39633165249152063 (STAR)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39628446967531209 (QSO at z=2.84)

### SV1 dark (N=20); first 10:
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39628395528587194 (STAR, WD)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39627914815214333 (QSO at z=1.181)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39627914819409528 (QSO, BAL at z=2.179)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39628417116668690 (QSO at z=2.38)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39627158296986610 (QSO at z=1.98)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39627218212621778 (QSO at z=1.373)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39628151147466939 (QSO at z=3.04)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39632945178216893 (QSO at z=1.24)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39628522041380522 (QSO at z=2.4)
- https://www.legacysurvey.org/viewer-desi/desi-spectrum/dr1/targetid39632939855643261 (? looks bad, not sure)

### SV2 dark (N=6)
```
TARGETID	     SURVEY	PROGRAM	  Z_RR	       Z_QN	        Z_VI
39633318479659525	sv2	dark	 0.00095977    1.1393893	1.1399
39633362096229283	sv2	dark	-0.00028507    1.2618234	1.3022
39633297059350765	sv2	dark	-0.00031275    1.0668758	1.056
39633318471272704	sv2	dark	 0.00088319    1.2718421	1.2568
39633328860564489	sv2	dark	-0.00136077    1.2698301	1.2759
39633368509318861	sv2	dark	-5.000e-05     2.8922458	2.895
```

In [ ]:
# some VI for sv2-dark (pasted table above):
t_vi = Table()
t_vi['TARGETID'] = T_qsom_i['TARGETID'][is_star&hi_conf_95]
t_vi['SURVEY'] = T_qsom_i['SURVEY'][is_star&hi_conf_95]
t_vi['PROGRAM'] = T_qsom_i['PROGRAM'][is_star&hi_conf_95]
t_vi['Z_RR'] = T_qsom_i['Z_RR'][is_star&hi_conf_95]
t_vi['Z_QN'] = T_qsom_i['Z_QN'][is_star&hi_conf_95]
t_vi['Z_VI'] = [1.1399, 1.3022, 1.0560, 1.2568, 1.2759, 2.895]

t_vi